In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys


import tflearn.data_utils as du
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix

DIRECTORY = r"C:\Users\rassa\Documents\Machine Learning A-Z  Udemy\Arabic Handwritten Characters Dataset"



Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
#Load the dataset
#header = 0 because
#this creates a pandas dataframe (basically a table)
train_data = pd.read_csv(DIRECTORY +  '/csvTrainImages 13440x1024.csv', header = None)
train_label = pd.read_csv(DIRECTORY + '/csvTrainLabel 13440x1.csv', header = None)

test_label = pd.read_csv(DIRECTORY +  '/csvTestLabel 3360x1.csv', header = None)


#what happens if we take off header = none ?
#Row number(s) to use as the column names, and the start of the data. Default behavior is to infer the column names: if no 
#names are passed the behavior is identical to header=0 and column names are inferred from the first line of the file, 
#if column names are passed explicitly then the behavior is identical to header=None.
#notice if we do this we have one row less
test_data = pd.read_csv(DIRECTORY +   '/csvTestImages 3360x1024.csv')
print(test_data)

test_data = pd.read_csv(DIRECTORY +   '/csvTestImages 3360x1024.csv', header = None)
print(test_data)
type(train_data)

      0  0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9  ...  0.919  0.920  \
0     0    0    0    0    0    0    0    0    0    0  ...      0      0   
1     0    0    0    0    0    0    0    0    0    0  ...      0      0   
2     0    0    0    0    0    0    0    0    0    0  ...      0      0   
3     0    0    0    0    0    0    0    0    0    0  ...      0      0   
4     0    0    0    0    0    0    0    0    0    0  ...      0      0   
...  ..  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...    ...    ...   
3354  0    0    0    0    0    0    0    0    0    0  ...      0      0   
3355  0    0    0    0    0    0    0    0    0    0  ...      0      0   
3356  0    0    0    0    0    0    0    0    0    0  ...      0      0   
3357  0    0    0    0    0    0    0    0    0    0  ...      0      0   
3358  0    0    0    0    0    0    0    0    0    0  ...      0      0   

      0.921  0.922  0.923  0.924  0.925  0.926  0.927  0.928  
0         0      0      0      0    

pandas.core.frame.DataFrame

In [3]:
train_data = train_data.iloc[:,:].values.astype('float32')
train_label = train_label.iloc[:,:].values.astype('int32')-1
test_data = test_data.iloc[:,:].values.astype('float32')
test_label = test_label.iloc[:,:].values.astype('int32')-1


In [4]:
train_label = du.to_categorical(train_label,28)

In [5]:
train_data = train_data/255
test_data = test_data/255

In [6]:
train_data = train_data.reshape([-1, 32, 32, 1])
test_data = test_data.reshape([-1, 32, 32, 1])

In [7]:
train_data, mean1 = du.featurewise_zero_center(train_data)
test_data, mean2 = du.featurewise_zero_center(test_data)

In [8]:

recognizer = Sequential()

recognizer.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (32,32,1)))

recognizer.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
recognizer.add(Dropout(0.25))

recognizer.add(Flatten())
recognizer.add(Dense(units = 256, input_dim = 1024, activation = 'relu'))
recognizer.add(Dense(units = 256, activation = "relu"))
recognizer.add(Dropout(0.5))
recognizer.add(Dense(28, activation = "softmax"))

In [9]:
recognizer.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2097408   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0

In [10]:
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

In [11]:
recognizer.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [12]:
datagen = ImageDataGenerator(
        featurewise_center=False, 
        samplewise_center=False,  
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=10,
        zoom_range = 0.1,  
        width_shift_range=0.1, 
        height_shift_range=0.1,
        horizontal_flip=False,
        vertical_flip=False)

In [13]:
datagen.fit(train_data)

In [14]:
recognizer.fit_generator(datagen.flow(train_data,train_label, batch_size=100),
                             epochs = 30, verbose = 2, steps_per_epoch=train_data.shape[0] // 100)

C:\Users\rassa\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training_v1.py:1240: UserWarning: `model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`model.fit_generator` is deprecated and '


Epoch 1/30
134/134 - 24s - loss: 2.5471 - acc: 0.2332
Epoch 2/30
134/134 - 32s - loss: 1.6169 - acc: 0.4556
Epoch 3/30
134/134 - 33s - loss: 1.2334 - acc: 0.5668
Epoch 4/30
134/134 - 30s - loss: 1.0065 - acc: 0.6458
Epoch 5/30
134/134 - 27s - loss: 0.8741 - acc: 0.6918
Epoch 6/30
134/134 - 25s - loss: 0.7807 - acc: 0.7231
Epoch 7/30
134/134 - 24s - loss: 0.7262 - acc: 0.7395
Epoch 8/30
134/134 - 24s - loss: 0.6648 - acc: 0.7649
Epoch 9/30
134/134 - 25s - loss: 0.6300 - acc: 0.7782
Epoch 10/30
134/134 - 26s - loss: 0.5920 - acc: 0.7918
Epoch 11/30
134/134 - 24s - loss: 0.5739 - acc: 0.8002
Epoch 12/30
134/134 - 25s - loss: 0.5415 - acc: 0.8111
Epoch 13/30
134/134 - 25s - loss: 0.5270 - acc: 0.8204
Epoch 14/30
134/134 - 23s - loss: 0.4910 - acc: 0.8292
Epoch 15/30
134/134 - 23s - loss: 0.4750 - acc: 0.8316
Epoch 16/30
134/134 - 24s - loss: 0.4625 - acc: 0.8432
Epoch 17/30
134/134 - 26s - loss: 0.4479 - acc: 0.8420
Epoch 18/30
134/134 - 23s - loss: 0.4415 - acc: 0.8507
Epoch 19/30
134/134

In [15]:
predictions = recognizer.predict(test_data)
predictions = np.argmax(predictions,axis = 1)

C:\Users\rassa\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


In [16]:
cm = confusion_matrix(test_label, predictions)

In [17]:
accuracy = sum(cm[i][i] for i in range(28)) / test_label.shape[0]
print("accuracy = " + str(accuracy))

accuracy = 0.924702380952381


In [18]:
print(cm)

[[120   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0 119   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   1]
 [  0   0  68  52   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   3 115   0   0   0   0   0   0   0   0   1   0   0   0   0   0
    0   0   0   0   0   0   1   0   0   0]
 [  0   0   0   0 117   2   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   1]
 [  0   0   0   1  22  92   2   0   0   0   0   0   0   0   0   0   0   3
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   1   2 116   0   0   0   0   0   0   0   0   0   0   0
    1   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0 116   3   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   1   0]
 [  0   0   0   0   0   0   0   3 113   0   1   0   0   0   0   